# MNIST GAN from Scratch (with GPU Acceleration)
## Objective:
To demonstrate the power of GPU-accelerated parallel processing using a Generative Adversarial Network (GAN) trained on the MNIST dataset. We'll generate synthetic images of handwritten digits.

## Setup
We will use TensorFlow which automatically utilizes the GPU if any

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Dense, Reshape, Flatten, LeakyReLU
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt